### Import modules

In [1]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, helpers
import boto3
from botocore.config import Config
import json

### Initialize and configure Boto Client for AOSS

In [ ]:
boto_config = Config(
    region_name = 'us-west-2',
    signature_version = 'v4',
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)

In [ ]:
client = boto3.client("opensearchserverless",config=boto_config)

### Define encryption policy

In [ ]:
policy = """{
                "Rules": [
                    {
                        "Resource": [
                            "collection/oscars-collection"
                        ],
                        "ResourceType": "collection"
                    }
                ],
                "AWSOwnedKey": true
            }"""

try:
    response = client.create_security_policy(
        description="oscars collection encryption security policy",
        name="oscars-encryption-policy",
        policy=policy,
        type="encryption",
    )
    print(json.dumps(response, indent=2))
except Exception as ex:
    print(ex)

### Define network policy

In [ ]:
policy = """[
                {
                    "Rules": [
                        {
                            "Resource": [
                                "collection/oscars-collection"
                            ],
                            "ResourceType": "collection"
                        }
                    ],
                    "AllowFromPublic": true
                }
            ]"""

try:
    response = client.create_security_policy(
        description="oscars collection network security policy",
        name="oscars-network-policy",
        policy=policy,
        type="network",
    )
    print(json.dumps(response, indent=2))
except Exception as ex:
    print(ex)

### Define data access policy

In [ ]:
#replace the IAM principle with your own
policy = """[
                {
                    "Rules": [
                        {
                            "Resource": [
                                "collection/*"
                            ],
                            "Permission": [
                                "aoss:*"
                            ],
                            "ResourceType": "collection"
                        },
                        {
                            "Resource": [
                                "index/*/*"
                            ],
                            "Permission": [
                                "aoss:*"
                            ],
                            "ResourceType": "index"
                        }
                    ],
                    "Principal": [
                        "arn:aws:iam::XXXX:user/YYYY" 
                    ],
                    "Description": "Rule 1"
                }
            ]"""

try:
    response = client.create_access_policy(
        description="oscars collection data access policy",
        name="oscars-data-access-policy",
        policy=policy,
        type="data"
    )
    print(json.dumps(response, indent=2))
except Exception as ex:
    print(ex)

### Create the collection

In [ ]:
# create vector search collection
try:
    response = client.create_collection(
        description="oscars collection",
        name="oscars-collection",
        type="VECTORSEARCH"
    )
    print(json.dumps(response, indent=2))
except Exception as ex:
    print(ex) 

### Initialize and configure OpenSearch client

In [ ]:
host = "" #replace this with the value from the AWS Management Console
region = "us-west-2"
service = "aoss"
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region, service)

In [ ]:
client = OpenSearch(
    hosts = [{"host": host, "port": 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 20
)

### Create an index

In [ ]:
index_name = "oscars-index"
index_body = {
    "mappings": {
        "properties": {
            "nominee_text": {"type": "text"},
            "nominee_vector": {
                "type": "knn_vector",
                "dimension": 4096,
                "method": {
                    "engine": "nmslib",
                    "space_type": "cosinesimil",
                    "name": "hnsw",
                    "parameters": {"ef_construction": 512, "m": 16},
                },
            },
        }
    },
    "settings": {
        "index": {
            "number_of_shards": 2,
            "knn.algo_param": {"ef_search": 512},
            "knn": True,
        }
    },
}

try:
    response = client.indices.create(index_name, body=index_body)
    print(json.dumps(response, indent=2))
except Exception as ex:
    print(ex)

### Verify the index

In [ ]:
try:
    response = client.indices.get("oscars-index")
    print(json.dumps(response, indent=2))
except Exception as ex:
    print(ex)